In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import cv2    
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from keras import regularizers
from keras.layers import Conv2D, Flatten, MaxPooling1D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import os
import base64
from keras.applications.xception import Xception
plt.style.use('ggplot')

%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Get the version of Tensorflow

In [5]:
import tensorflow as tf
print(tf.__version__)

# Data Exploration
We will be using the CelebA Dataset, which includes images of 178 x 218 px. Below is an example of how the pictures looks like.

In [6]:
# set variables 
main_folder = ''
images_folder = main_folder + '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/'

EXAMPLE_PIC = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg'

TRAINING_SAMPLES = 5000
VALIDATION_SAMPLES = 2000
TEST_SAMPLES = 500
IMG_WIDTH = 178
IMG_HEIGHT = 218
BATCH_SIZE=32

In [7]:
# import the data set that include the attribute for each picture
df_attr = pd.read_csv("/kaggle/input/celeba-dataset/list_attr_celeba.csv")
df_attr.set_index('image_id', inplace=True)
df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
df_attr.shape

In [8]:
# List of available attributes
for i, j in enumerate(df_attr.columns):
    print(i, j)

In [9]:
# plot picture and attributes
img = load_img(EXAMPLE_PIC)
plt.grid(False)
plt.imshow(img)
df_attr.loc[EXAMPLE_PIC.split('/')[-1]][['Smiling','Male','Young']] #some attributes

# Training, Validation and Test
The recommended partitioning of images into training, validation, testing of the data set is:

1-162770 are training 162771-182637 are validation 182638-202599 are testing The partition is in file list_eval_partition.csv

Due time execution, by now we will be using a reduced number of images:

Training 5000 images Validation 2000 images Test 500 Images

In [10]:
df_partition = pd.read_csv('../input/celeba-dataset/list_eval_partition.csv')
df_partition.head()

In [11]:
df_partition['partition'].value_counts().sort_index()

# Join the partition and the attributes in the same data frame

In [12]:
# join the partition with the attributes
df_partition.set_index('image_id', inplace=True)
df_par_attr = df_partition.join(df_attr['Male'], how='inner')
df_par_attr = df_par_attr.join(df_attr['Smiling'], how='inner')
df_par_attr = df_par_attr.join(df_attr['Young'], how='inner')
df_par_attr.head()

In [13]:
def load_reshape_img(fname):
    img = load_img(fname)
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x


def generate_df(partition, attr, num_samples):
    
    df_ = df_par_attr[(df_par_attr['partition'] == partition) 
                           & (df_par_attr[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      df_par_attr[(df_par_attr['partition'] == partition) 
                                  & (df_par_attr[attr] == 1)].sample(int(num_samples/2))])


    if partition != 2:
        x_ = np.array([load_reshape_img(images_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 218, 178, 3)
        y_ = np_utils.to_categorical(df_[attr],2)

    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(images_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

# Data Augmentation
This is how an image will look like after data augmentation (based in the giving parameters below).

In [14]:
# Generate image generator for data augmentation
datagen =  ImageDataGenerator(
  #preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

# load one image and reshape
img = load_img(EXAMPLE_PIC)
x = img_to_array(img)/255.
x = x.reshape((1,) + x.shape)

# plot 10 augmented images of the loaded iamge
plt.figure(figsize=(20,10))
plt.suptitle('Data Augmentation', fontsize=28)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.subplot(3, 5, i+1)
    plt.grid(False)
    plt.imshow( batch.reshape(218, 178, 3))
    
    if i == 9:
        break
    i += 1
    
plt.show()

The result is a new set of images with modifications from the original one, that allows to the model to learn from these variations in order to take this kind of images during the learning process and predict better never seen images.

# Step 4: Build the Model
## 4.1. Set the Model

In [15]:
x_train, y_train = generate_df(0, 'Male', TRAINING_SAMPLES)
x_valid, y_valid = generate_df(1, 'Male', VALIDATION_SAMPLES)

train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_generator = train_datagen.flow(
x_train, y_train,
batch_size=BATCH_SIZE
)

valid_generator = train_datagen.flow(
x_valid, y_valid,
batch_size=BATCH_SIZE
)


epochs = 15
learning_rate = 0.0001
batch_size = 128
weights = os.path.join('', 'weights.h5')
callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)]

base_model = Xception(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False) # Average pooling reduces output dimensions
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

# ------ TRAINING ------
model.fit_generator(train_generator,steps_per_epoch=len(x_train)/batch_size,validation_data=valid_generator
                    ,callbacks=callbacks,epochs=epochs,verbose=1)

In [16]:
x_train, y_train = generate_df(0, 'Smiling', TRAINING_SAMPLES)
x_valid, y_valid = generate_df(1, 'Smiling', VALIDATION_SAMPLES)

train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_generator = train_datagen.flow(
x_train, y_train,
batch_size=BATCH_SIZE
)

valid_generator = train_datagen.flow(
x_valid, y_valid,
batch_size=BATCH_SIZE
)



epochs = 15
learning_rate = 0.0001
batch_size = 128
weights = os.path.join('', 'weights.h5')
callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)]

base_model = Xception(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False) # Average pooling reduces output dimensions
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
model_smile = Model(inputs=base_model.input, outputs=predictions)    
model_smile.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

# ------ TRAINING ------
model_smile.fit_generator(train_generator,steps_per_epoch=len(x_train)/batch_size,validation_data=valid_generator
                    ,callbacks=callbacks,epochs=epochs,verbose=1)

# Model Evaluation

In [17]:
x_test, y_test = generate_df(2, 'Male', TEST_SAMPLES)

In [18]:
x_test_smile, y_test_smile = generate_df(2, 'Smiling', TEST_SAMPLES)

In [19]:
#x_test_young, y_test_young = generate_df(2, 'Young', TEST_SAMPLES)

In [20]:
if os.path.isfile(weights):
    model.load_weights(weights)
    model_smile.load_weights(weights)
    #model_young.load_weights(weights)

In [21]:
model_predictions = [np.argmax(model.predict(feature)) for feature in x_test ]

In [22]:
model_predictions_smile = [np.argmax(model_smile.predict(feature)) for feature in x_test_smile ]

In [23]:
#model_predictions_young = [np.argmax(model_young.predict(feature)) for feature in x_test_young ]

In [24]:
test_accuracy = 100 * np.sum(np.array(model_predictions)==y_test) / len(model_predictions)
print('Model Evaluation ')
print('Test accuracy: %.4f%%' % test_accuracy)
print('f1_score:', f1_score(y_test_smile, model_predictions_smile))

In [25]:
test_accuracy = 100 * np.sum(np.array(model_predictions_smile)==y_test_smile) / len(model_predictions_smile)
print('Model Evaluation')
print('Test accuracy: %.4f%%' % test_accuracy)
print('f1_score:', f1_score(y_test_smile, model_predictions_smile))

In [26]:
#test_accuracy = 100 * np.sum(np.array(model_predictions_young)==y_test_young) / len(model_predictions_young)
#print('Model Evaluation')
#print('Test accuracy: %.4f%%' % test_accuracy)
#print('f1_score:', f1_score(y_test_smile, model_predictions_smile))